## (1) 神经网络结构设计
我们将使用一个单层前馈神经网络（单隐层），用于对鸢尾花数据集进行三分类任务。

- 输入层：4个特征
- 输出层：3个类别，对应鸢尾花的三个种类（使用Softmax输出）
- 激活函数：Softmax（用于多分类）
- 损失函数：交叉熵损失（CrossEntropyLoss）
- 优化器：Adam

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 载入数据
iris = load_iris()
X, y = iris.data, iris.target

# 数据标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转为Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# 定义模型
class SingleLayerNN(nn.Module):
    def __init__(self):
        super(SingleLayerNN, self).__init__()
        self.fc = nn.Linear(4, 3)

    def forward(self, x):
        return self.fc(x)

# 模型训练函数
def train_model(model, optimizer, loss_fn, X_train, y_train, X_test, y_test, epochs=100):
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fn(outputs, y_train)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_train).float().mean().item()
        train_accuracies.append(acc)

        # 测试阶段
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = loss_fn(test_outputs, y_test)
            test_losses.append(test_loss.item())
            _, test_pred = torch.max(test_outputs, 1)
            test_acc = (test_pred == y_test).float().mean().item()
            test_accuracies.append(test_acc)

    training_time = time.time() - start_time
    return train_losses, test_losses, train_accuracies, test_accuracies, training_time

# 初始化
model = SingleLayerNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# 训练
train_losses, test_losses, train_accs, test_accs, train_time = train_model(model, optimizer, loss_fn, X_train, y_train, X_test, y_test)

print(f"训练耗时: {train_time:.4f}秒")

# 画图
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.title("Cross Entropy Loss")

plt.subplot(1,2,2)
plt.plot(train_accs, label='Train Acc')
plt.plot(test_accs, label='Test Acc')
plt.legend()
plt.title("Accuracy")
plt.show()


## (3) 超参数调优与实验记录
通过尝试不同的学习率（如0.1, 0.01, 0.001）、优化器（如SGD, Adam）、训练轮数等参数组合，记录模型性能指标（准确率、损失、训练时间），选出最优方案。

In [ ]:
# 函数封装（复用）
def evaluate_model(model, X, y, loss_fn):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        loss = loss_fn(outputs, y)
        _, pred = torch.max(outputs, 1)
        acc = (pred == y).float().mean().item()
    return loss.item(), acc
